In [2]:
from dask.distributed import Client
import dask.array as da

In [17]:
import dask
#dask.config.config

In [21]:
from dask_cloudprovider.aws import EC2Cluster

check if we have sufficient permissions

In [22]:
#!aws s3 ls

In [18]:
#!aws ec2 describe-subnets

In [19]:
#!aws ec2 describe-instances

Create the cluster on AWS

In [24]:
cluster = EC2Cluster(region="us-east-1",
                     n_workers=4, 
                     worker_instance_type='t2.medium', 
                     scheduler_instance_type='t2.micro',
                 
                     security=False)

Creating scheduler instance
Created instance i-08135df9f291ae92f as dask-8ebd5e82-scheduler
Waiting for scheduler to run at 54.165.239.51:8786
Scheduler is running


C:\Users\sacha\anaconda3\envs\dask-cluster\lib\contextlib.py:142: UserWarning: Creating your cluster is taking a surprisingly long time. This is likely due to pending resources. Hang tight! 
  next(self.gen)


Creating worker instance
Creating worker instance
Creating worker instance
Creating worker instance
Created instance i-03c9b4d6297037fc2 as dask-8ebd5e82-worker-bb7cd4a3
Created instance i-098abc48343d3c49a as dask-8ebd5e82-worker-915e0641
Created instance i-043b13ec4d81eff4b as dask-8ebd5e82-worker-371184f4
Created instance i-0e356ebdca171aa83 as dask-8ebd5e82-worker-50be33ff


In [30]:
client = Client(cluster)
client

C:\Users\sacha\anaconda3\envs\dask-cluster\lib\site-packages\distributed\client.py:1606: VersionMismatchWarning: Mismatched versions found

+-------------+--------+-----------+---------+
| Package     | Client | Scheduler | Workers |
+-------------+--------+-----------+---------+
| cloudpickle | 3.1.0  | 3.0.0     | 3.0.0   |
| numpy       | 2.1.0  | 2.1.1     | 2.1.1   |
| toolz       | 1.0.0  | 0.12.0    | 0.12.0  |
+-------------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tcp://172.31.23.27:8786' processes=4 threads=8, memory=15.27 GiB>

In [31]:
import dask.array as da

a_da = da.ones(10, chunks=5)
a_da

dask.array<ones_like, shape=(10,), dtype=float64, chunksize=(5,), chunktype=numpy.ndarray>

In [32]:
a_da_sum = a_da.sum()
a_da_sum

dask.array<sum-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [33]:
a_da_sum.compute()

np.float64(10.0)

In [34]:
xd = da.random.normal(10, 0.1, size=(40_000, 40_000), chunks=(3000, 3000))
xd

dask.array<normal, shape=(40000, 40000), dtype=float64, chunksize=(3000, 3000), chunktype=numpy.ndarray>

In [35]:
%%time
xd = da.random.normal(10, 0.1, size=(30_000, 30_000), chunks=(3000, 3000))
yd = xd.mean(axis=0)
yd.compute()

CPU times: total: 15.6 ms
Wall time: 4.8 s


array([10.00011163, 10.0007226 , 10.00021809, ...,  9.99998737,
        9.99987812,  9.99977536])

In [36]:
client.close()

In [37]:
cluster.close()

Terminated dask-8ebd5e82-worker-371184f4 (i-043b13ec4d81eff4b)
Terminated dask-8ebd5e82-worker-50be33ff (i-0e356ebdca171aa83)
Terminated dask-8ebd5e82-worker-bb7cd4a3 (i-03c9b4d6297037fc2)
Terminated dask-8ebd5e82-worker-915e0641 (i-098abc48343d3c49a)
Terminated dask-8ebd5e82-scheduler (i-08135df9f291ae92f)
